In [1]:
import numpy as np
import math

In [2]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [3]:
def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
        
    if (all([r == math.inf for r in restrictions])):
        raise Exception("Linear program is unbounded.")

    row = restrictions.index(min(restrictions))
    return row, column


def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

def to_objective_function_value(c, solution):
    return sum(np.array(c) * np.array(solution))

def can_be_improved_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    return any([entry < 0 for entry in rhs_entries])

def get_pivot_position_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    min_rhs_value = min(rhs_entries)
    row = rhs_entries.index(min_rhs_value)
    
    columns = []
    for index, element in enumerate(tableau[row][:-1]):
        if element < 0:
            columns.append(index)
    columns_values = [tableau[row][c] / tableau[-1][c] for c in columns]
    column_min_index = columns_values.index(min(columns_values))
    column = columns[column_min_index]

    return row, column

def dual_simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved_for_dual(tableau):
        pivot_position = get_pivot_position_for_dual(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [4]:
c = [1,1,3]
A = [
    [1,2,4],
    [1,1,2],
    [1,2,1]
]
b = [1,3,4]
solution = simplex(c, A, b)
print('solution: ', solution)

primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

solution:  [1.0, 0, 0]
Primal:  1.0


In [4]:
c = [2, 1, 0, 0, 0]
A = [
    [1, -1, 1, 0, 0],
    [ 3, -1, 0, 1, 0],
    [ -1, 2, 0, 0, 1]
]
b = [4, 18, 6]
solution = simplex(c, A, b)
print('solution: ', solution)

primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

solution:  [8.4, 7.199999999999999, 2.8, 0, 0]
Primal:  24.0


In [5]:
c = [3, 4, 0, 0]
A = [
    [-3, 4, 1, 0],
    [ -2, 1, 0, 1]
]
b = [12, 2]
solution = simplex(c, A, b)
print('solution: ', solution)

Exception: Linear program is unbounded.

In [6]:
c = [-3, -2, 0, 0]
A = [
    [2, 1, 1, 0],
    [2, 3, 0, 1],
]
b = [12, 16]
solution = simplex(c, A, b)
print('solution: ', solution)

primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

solution:  [0, 0, 12, 16]
Primal:  0


In [7]:
c = [2, 1, 0, 0, 0]
A = [
    [1, -1, 1, 0, 0],
    [ 3, -1, 0, 1, 0],
    [ -1, 2, 0, 0, 1]
]
b = [4, 18, 6]
solution = dual_simplex(c, A, b)
print('solution: ', solution)

dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

solution:  [0, 0, 4, 18, 6]
Dual:  0


In [8]:
c = [12, 16, 0, 0]
A = [
    [-2, -2, 1, 0],
    [-1, -3, 0, 1]
]
b = [-3, -2]
solution = dual_simplex(c, A, b)
print('solution: ', solution)

dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

solution:  [1.25, 0.25, 0, 0]
Dual:  19.0
